In [1]:
import numpy as np
import pandas as pd
import copy
from scipy.stats import t, chi2

# Event codes
ARRIVAL=0
DEPARTURE=1

# Patient status codes
PENDING=0
ADMITTED=1
RELOCATED=2
REJECTED=3
DISCHARGED=4

# Simulation parameters
SIM_TIME = 30   # Simulation time in days


In [3]:
wards = dict(
    capacities = [55, 40, 30, 20, 20, 0],
    lams = [14.5, 11.0, 8.0, 6.5, 5.0, 13.0],
    mu_invs = [2.9, 4.0, 4.5, 1.4, 3.9, 2.2],
    urgency_points = [7, 5, 2, 10, 5, 0],
    occupancy = [0, 0, 0, 0, 0, 0],
)

relocation_probability = np.array([
    [0.0, 0.05, 0.10, 0.05, 0.80, 0.00],
    [0.2, 0, 0.50, 0.15, 0.15, 0.00],
    [0.30, 0.20, 0, 0.20, 0.30, 0.00],
    [0.35, 0.30, 0.05, 0, 0.3, 0.00],
    [0.20, 0.10, 0.60 ,0.10, 0, 0.00],
    [0.20, 0.20, 0.20, 0.20, 0.20 ,0]
    ])

**Simulation Setup**

In [4]:
def simulatation_setup(wards, ar_sampler, los_sampler, sim_time=SIM_TIME):
    """
    Keyword arguments:
    wards -- dictionary containing ward parameters:
        capacities -- array of ward capacities
        lams -- array of arrival rates
        mu_invs -- array of inverse lengths of stay rates
        urgency_points -- array of urgency points
        occupancy -- array of current ward occupancies

    ar_sampler -- arrival time sampler, takes lambda as input
    los_sampler -- length of stay sampler, takes 1/mu as input
    sim_time -- simulation time in months

    Returns:
    events -- dictionary of simulation events:
        time -- list of event times
        ward -- list of ward indices
        event -- list of event types
        ID -- list of patient IDs
        status -- list of patient statuses
    """

    # Precompute arrival and departure times for each ward
    events = dict( time=[], ward=[], event=[], ID=[] )
    patient_id = 0
    for i, (lam, mu_inv) in enumerate(zip(wards['lams'], wards['mu_invs'])):
        
        # Sample patients for ward
        clock = 0
        while clock < sim_time:

            # Sample arrival time
            clock += ar_sampler(lam)

            # Add patient to event list if arrival time is before simulation end
            if clock <= sim_time:
                events['time'] += [clock, clock+los_sampler(mu_inv)]
                events['ID'] += [patient_id]*2
                events['ward'] += [i]*2
                events['event'] += [ARRIVAL, DEPARTURE]
                patient_id += 1


    # Sort events by time
    idx = np.argsort(events['time'])
    for key in events.keys():
        events[key] = [events[key][i] for i in idx]

    # Add simulation lists
    N = len(idx)
    events['status'] = [PENDING]*N
    events['new_ward'] = [None]*N

    return events


**Inspect the Event List**

In [5]:
ar_sampler = lambda lam: np.random.exponential(1/lam)
los_sampler = lambda mu_inv: np.random.exponential(mu_inv)
events = simulatation_setup(wards, ar_sampler, los_sampler)
df = pd.DataFrame(events)
print(df)

           time  ward  event    ID  status new_ward
0      0.006138     2      0   844       0     None
1      0.033458     5      0  1437       0     None
2      0.035700     5      0  1438       0     None
3      0.055619     5      0  1439       0     None
4      0.060836     5      0  1440       0     None
...         ...   ...    ...   ...     ...      ...
3721  42.357472     1      1   829       0     None
3722  42.496211     1      1   718       0     None
3723  42.496941     1      1   830       0     None
3724  43.176215     2      1  1017       0     None
3725  45.219268     1      1   805       0     None

[3726 rows x 6 columns]


**Simulation Loop**

In [6]:
def simulate(events, wards, relocation_probability):

    # Clear event statuses
    events['status'] = [PENDING]*len(events['time'])

    # Simulate loop
    for idx, (ward, event, id) in enumerate(zip(events['ward'], events['event'], events['ID'])):

        if event == ARRIVAL:

            # Admit patient
            if wards['occupancy'][ward] < wards['capacities'][ward]:
                wards['occupancy'][ward] += 1
                events['status'][idx] = ADMITTED

            else:
                # Relocate patient
                events['status'][idx] = RELOCATED
                departure_idx = events['ID'].index(id, idx+1)
                new_ward = np.random.choice(len(wards['capacities']), p=relocation_probability[ward])
                if wards['occupancy'][new_ward] < wards['capacities'][new_ward]:
                    wards['occupancy'][new_ward] += 1
                    
                    # Update ward of departure event
                    events['ward'][idx] = events['ward'][departure_idx] = new_ward
                                                        
                # Reject patient
                else:
                    events['status'][idx] = events['status'][departure_idx] = REJECTED


        # Discharge patient
        elif events['status'][idx] != REJECTED:
            wards['occupancy'][ward] -= 1
            events['status'][idx] = DISCHARGED

In [8]:
for _ in range(1000):
    events = simulatation_setup(wards, ar_sampler, los_sampler)
    simulate(events, wards, relocation_probability)

**Confidence Intervals**

In [7]:
# alpha = 0.05

# def control_variate(X, U, mu=0.5):
#     c = -np.cov(X, U)[0,1]/np.var(U)
#     return X + c*(U - mu)

# X = np.array(events['faith'])==REJECTED
# U = np.array(events['U'])
# Z = control_variate(X, U)
# N = len(X)

# muX, varX = X.mean(), X.var()
# muZ, varZ = Z.mean(), Z.var()

# muX_CI = muX + t.ppf([alpha/2, 1-alpha/2], N-1) * np.sqrt(varX/N)
# muZ_CI = muZ + t.ppf([alpha/2, 1-alpha/2], N-1) * np.sqrt(varZ/N)
# varX_CI = (N-1)*varX / chi2.ppf([1-alpha/2, alpha/2], N-1)
# varZ_CI = (N-1)*varZ / chi2.ppf([1-alpha/2, alpha/2], N-1)

# print(f"muX = {muX:.3f} ({muX_CI[0]:.3f}, {muX_CI[1]:.3f})")
# print(f"muZ = {muZ:.3f} ({muZ_CI[0]:.3f}, {muZ_CI[1]:.3f})")